In [1]:
import os
import sys
import pandas
import hashlib
import hmac
import requests
import datetime
import time
import json
import configparser
import traceback

In [2]:
# APIアカウント
config = configparser.ConfigParser()
config.read('./config.ini')

BF_ACCESSKEY = config['BITFLYER']['ACCESSKEY']
BF_SECRETKEY = config['BITFLYER']['SECRETKEY']
CC_ACCESSKEY = config['COINCHECK']['ACCESSKEY']
CC_SECRETKEY = config['COINCHECK']['SECRETKEY']

TRADE_VOLUME = 0.01

In [3]:
def bf_api(method, path, param):
    base_url = "https://api.bitflyer.com"
    timestamp = str(datetime.datetime.today())
    body = json.dumps(param)
    message = timestamp + method + path + body if (param is not None) else timestamp + method + path
    signature = hmac.new(bytearray(BF_SECRETKEY.encode('utf-8')), message.encode('utf-8') , digestmod = hashlib.sha256 ).hexdigest()
    headers = {
        'ACCESS-KEY' : BF_ACCESSKEY,
        'ACCESS-TIMESTAMP' : timestamp,
        'ACCESS-SIGN' : signature,
        'Content-Type' : 'application/json'
    }
    if method == 'POST':
        return requests.post( base_url + path , data = body , headers = headers)
    elif method == 'GET':
        return requests.get( base_url + path , headers = headers)
    
def bf_getBalance():
    return bf_api("GET", "/v1/me/getcollateral", None).json()
    
def bf_getBalance_hist():
    return bf_api("GET", "/v1/me/getcollateralhistory", None).json()
    
def bf_getPositions():
    return bf_api("GET", "/v1/me/getpositions?product_code=FX_BTC_JPY", None).json()

def bf_checkLTP():
    return int(bf_api("GET", "/v1/ticker?product_code=FX_BTC_JPY", None).json()['ltp'])
    
def bf_sell(volume):
    param = {
        "product_code": "FX_BTC_JPY",
        "child_order_type": "MARKET",
        "side": "SELL",
        "size": volume,
        "minute_to_expire": 30,
        "time_in_force": "GTC"
    }
    return bf_api("POST", "/v1/me/sendchildorder", param).json()
    
def bf_buy(volume):
    param = {
        "product_code": "FX_BTC_JPY",
        "child_order_type": "MARKET",
        "side": "BUY",
        "size": volume,
        "minute_to_expire": 30,
        "time_in_force": "GTC"
    }
    return bf_api("POST", "/v1/me/sendchildorder", param).json()
    
def bf_getexecutions(order_id):
    return bf_api("GET", "/v1/me/getexecutions?product_code=FX_BTC_JPY&child_order_acceptance_id={}".format(order_id), None).json()

In [4]:
class Coincheck:
    def __init__(self, access_key, secret_key, url='https://coincheck.com'):
        self.access_key = access_key
        self.secret_key = secret_key
        self.url = url

    def get(self, path, params=None):
        if params != None:
            params = json.dumps(params)
        else:
            params = ''
        nonce = str(int(time.time()))
        message = nonce + self.url + path + params
        signature = self.getSignature(message)
        return requests.get(
            self.url+path,
            headers=self.getHeader(self.access_key, nonce, signature)
        ).json()

    def post(self, path, params):
        params = json.dumps(params)
        nonce = str(int(time.time()))
        message = nonce + self.url + path + params
        signature = self.getSignature(message)
        return requests.post(
            self.url+path,
            data=params,
            headers=self.getHeader(self.access_key, nonce, signature)
        ).json()

    def delete(self, path):
        nonce = str(int(time.time()))
        message = nonce + self.url + path
        signature = self.getSignature(message)
        return requests.delete(
            self.url+path,
            headers=self.getHeader(self.access_key, nonce, signature)
        ).json()

    def getSignature(self, message):
        signature = hmac.new(
            bytes(self.secret_key.encode('ascii')),
            bytes(message.encode('ascii')),
            hashlib.sha256
        ).hexdigest()

        return signature

    def getHeader(self, access_key, nonce, signature):
        headers = {
            'ACCESS-KEY': access_key,
            'ACCESS-NONCE': nonce,
            'ACCESS-SIGNATURE': signature,
            'Content-Type': 'application/json'
        }
        return headers
    
    def checkLTP(self):
        return int(self.get("/api/ticker")['last'])

    def buy(self, jpy_volume):
        params = {
            'order_type': "market_buy",
            'pair': "btc_jpy",
            'market_buy_amount': jpy_volume
        }
        return self.post("/api/exchange/orders", params)

    def sell(self, volume):
        params = {
            'order_type': "market_sell",
            'pair': "btc_jpy",
            'amount': volume
        }
        return self.post("/api/exchange/orders", params)
    
    def getexecutions(self):
        return self.get("/api/exchange/orders/transactions")
    

In [ ]:
gap_hist=[]
status = {
    "position": "init",
    "sell_gap": -1,
    "cc_volume": 0
}

cc = Coincheck(CC_ACCESSKEY, CC_SECRETKEY)
gap_hist.insert(0, bf_checkLTP()-cc.checkLTP())
gap_ave = int(sum(gap_hist)/len(gap_hist))

with open('./data/trade.txt', mode='a') as trade:
    while True:
        try:
            dd = datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")
            bf_ltp = bf_checkLTP()
            # cc_ltp = cc.checkLTP()
            ccticker = cc.get('/api/trades?pair=btc_jpy')['data']
            cc_ltp = ccticker[0]['rate']
            ### BFFXの売り、CCの買い
            if ((bf_ltp-cc_ltp) - gap_ave >= 1000) and status['position'] == "none":
                trade.write("{}, {}, {}, {}, {}, {}\n".format(dd, bf_ltp, cc_ltp, bf_ltp-cc_ltp, gap_ave, (bf_ltp-cc_ltp) - gap_ave))
                jpy_volume = int(cc_ltp * TRADE_VOLUME)
                cc_executions = cc.buy(jpy_volume)
                bf_executions = bf_sell(TRADE_VOLUME)
                time.sleep(5)
                # BFFX
                if "child_order_acceptance_id" in bf_executions:
                    orders = bf_getexecutions(bf_executions["child_order_acceptance_id"])
                    for o in orders:
                        trade.write("SELL, BFFX, {}, {}, {}\n".format(o['price'], o['size'], bf_executions["child_order_acceptance_id"]))
                # CC
                if "id" in cc_executions:
                    orders = cc.getexecutions()
                    for o in orders['transactions']:
                        if o['order_id'] == cc_executions['id']:
                            trade.write("BUY, CC, {}, {}, {}\n".format(o['rate'], o['funds']['btc'], cc_executions["id"]))
                time.sleep(1)
                status['cc_volume'] = cc.get("/api/accounts/balance")['btc']
                status['position'] = "sell"
                status['sell_gap'] = (bf_ltp-cc_ltp)
                trade.flush()

            ### 逆取引
            if (((bf_ltp-cc_ltp) - gap_ave <= -1000) or ((status['sell_gap'] - (bf_ltp-cc_ltp))>=3000) ) and status['position'] == "sell":
                trade.write("{}, {}, {}, {}, {}, {}\n".format(dd, bf_ltp, cc_ltp, bf_ltp-cc_ltp, gap_ave, (bf_ltp-cc_ltp) - gap_ave))
                cc_executions = cc.sell(status['cc_volume'])
                bf_executions = bf_buy(TRADE_VOLUME)
                time.sleep(5)
                # BFFX
                if "child_order_acceptance_id" in bf_executions:
                    orders = bf_getexecutions(bf_executions["child_order_acceptance_id"])
                    for o in orders:
                        trade.write("BUY, BFFX, {}, {}, {}\n".format(o['price'], o['size'], bf_executions["child_order_acceptance_id"]))
                # CC
                if "id" in cc_executions:
                    orders = cc.getexecutions()
                    for o in orders['transactions']:
                        if o['order_id'] == cc_executions['id']:
                            trade.write("SELL, CC, {}, {}, {}\n".format(o['rate'], o['funds']['btc'], cc_executions["id"]))
                trade.write("--------------------\n")
                status['position'] = "none"
                trade.flush()

            ### 後処理
            gap_hist.insert(0, bf_ltp-cc_ltp)
            gap_hist = gap_hist[:400] # 1分30回
            gap_ave = int(sum(gap_hist)/len(gap_hist))
            # f.write("{}, {}, {}, {}, {}\n".format(dd, bf_ltp, cc_ltp, bf_ltp-cc_ltp, gap_ave))
            # f.flush()

            ### 初期化
            if status['position'] == "init" and len(gap_hist)>300:
                status['position'] = "none"

            time.sleep(2)
        except Exception as e:
            trade.write(traceback.format_exc())
            trade.write("\n")
            trade.write("error\n")
            trade.flush()
            break

In [10]:
print(bf_checkLTP())
print(cc.checkLTP())

1219952
1211346
